Mengimpor library yang diperlukan

In [1]:
import pandas as pd
import tensorflow as tf
import requests

Membaca data yang digunakan untuk proses pengujian model

In [2]:
depression_df = pd.read_csv("./data/combined_sentiment_data.csv")
depression_df.head()

,sentence,sentiment
0,so there is no way for me to plug it in here i...,0
1,good case excellent value,1
2,great for the jawbone,1
3,tied to charger for conversations lasting more...,0
4,the mic is great,1


Membuat proses prediksi pada model. Proses ini dimulai dengan mengubah bentuk data dari csv ke dalam bentuk yang dapat diterima oleh serving model. Kemudian, data tersebut diteruskan ke model yang telah di-deploy pada Railway. Proses prediksi dilakukan pada salah satu baris data yang dipilih secara acak.

In [11]:
def create_tf_example(text):
    feature = {
        "sentence": tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[text.encode("utf-8")])
        ),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()


def get_prediction(text):
    serialized_text = create_tf_example(text)
    base64_encoded = tf.io.encode_base64(serialized_text).numpy().decode("utf-8")

    url = "https://dzulfikri-sentiment-sentence.up.railway.app/v1/models/sentence-sentiment:predict"
    data = {"instances": [{"b64": base64_encoded}]}
    response = requests.post(url, json=data)

    return response.json()


random_test_text = depression_df.sample(1).iloc[0]
prediction = get_prediction(random_test_text["sentence"]).get("predictions")[0]

if prediction:
    value = prediction[0]
    result = "positif" if value > 0.5 else "negatif"
    print(f"Text: {random_test_text['sentence']}")
    print(f"Actual: {random_test_text['sentiment']}")
    print(f"Prediction: {result} ({value})")
else:
    print("Failed to get prediction")

Text: the phone was unusable and was not new 
Actual: 0
Prediction: negatif (8.24413291e-05)
